#### preprocesss the csv files


In [6]:
import torch
import numpy as np

# Load the .pt file NOTE: process train/valid/test each time 
pt_tensor = torch.load('train.pt')
pt_tensor = torch.load('valid.pt')
pt_tensor = torch.load('test.pt')

# Convert to NumPy array
data = np.array(pt_tensor['edge'])

print(data.shape)

np.save("collab_train.npy", data)
np.save("collab_valid.npy", data)
np.save("collab_test.npy", data)

data_array = np.load("collab_train.npy")
data_array = np.load("collab_valid.npy")
data_array = np.load("collab_test.npy")

# find the max user and item idx
max_item_idx = np.max(data_array[:, 1])
max_user_idx = np.max(data_array[:, 0])
print(max_item_idx,max_user_idx, data_array.shape[0])
data_array = data_array.astype(int)

ones_column = np.ones((data_array.shape[0], 1))
new_data_array = np.hstack((data_array, ones_column))

condition = (new_data_array[:, 0] < 235868) & (new_data_array[:, 1] < 235868) #& (raw_array[:, 2] == 1)
raw_10000 = new_data_array[condition]
raw_10000 = np.array(raw_10000)

condition = (new_data_array[:, 0] < 235868) & (new_data_array[:, 1] < 235868) & (new_data_array[:, 2] == 1)
raw_10000_edge = new_data_array[condition]
raw_10000_edge = np.array(raw_10000_edge)
raw_10000_edge.shape


(3031780, 2)
576287 576287 3031780


(507618, 3)

#### modify the row dataset

In [7]:
def add_disconnected_edge(total_edge_num, connected_edge):
    num_disconnected_edges = total_edge_num - connected_edge.shape[0]
    existing_edges_set = set(map(tuple, connected_edge))
    disconnected_edges = set()

    while len(disconnected_edges) < num_disconnected_edges:
        edge = tuple(np.random.randint(0, 235868, size=2, dtype=np.int32))
        edge_extend = edge + (1,)
        if edge[0] != edge[1] and edge_extend not in existing_edges_set:
            disconnected_edges.add(edge)

    # Convert disconnected_edges to a numpy array
    disconnected_edges = np.array(list(disconnected_edges), dtype=np.int32)

    # Create arrays for all original and disconnected edges
    disconnected_edges =  np.column_stack((disconnected_edges, -1 * np.ones(num_disconnected_edges, dtype=np.int32)))

    return disconnected_edges.astype(int)


In [8]:
total_disconnect_edges = add_disconnected_edge(len(raw_10000_edge) * 2, raw_10000_edge)

In [9]:
#NOTE:process train/valid/test seperately for each run 
total_trained_edge = np.vstack((raw_10000_edge,total_disconnect_edges)).astype(int)
total_valid_edge = np.vstack((raw_10000_edge,total_disconnect_edges)).astype(int)
total_test_edge =np.vstack((raw_10000_edge,total_disconnect_edges)).astype(int)

In [10]:
#save the pre processed dataset to npy form 
np.save("./collab_users_entry_train.npy", total_trained_edge)
np.save("./collab_users_entry_valid.npy", total_valid_edge)
np.save("./collab_users_entry_test.npy", total_test_edge)

In [11]:
total_edge = np.concatenate((total_trained_edge, total_valid_edge, total_test_edge), axis = 0)
sorted_indices = np.lexsort((total_edge[:, 1], total_edge[:, 0]))
sorted_edges = total_edge[sorted_indices]
print(sorted_edges.shape)
print(sorted_edges[:10])

(3045708, 3)
[[     0 131037     -1]
 [     0 131037     -1]
 [     0 131037     -1]
 [     1  31768     -1]
 [     1  31768     -1]
 [     1  31768     -1]
 [     1  78236     -1]
 [     1  78236     -1]
 [     1  78236     -1]
 [     2   1910     -1]]


In [12]:
#create graph with no edges
import numpy as np
G = sorted_edges[:]
for i in range(G.shape[0]): G[i,2] = 0
np.save("./collab_users_noedge.npy", G)

## original file starting now:

In [13]:
import numpy as np
import pickle
import scipy as sp
import scipy.sparse as spp

def load_sparse_matrix(filename):
    t = 0
    f = open(filename, 'r')
    f.readline()

    user_list = []
    item_list = []
    data_list = []
    count  = 0
    for line in f:
        word = line.split(',')
       
        rating = float(word[2])

        t += 1
        if t % 50000 == 0:
            print('.', end = '')
            if t % 2000000 == 0:
                print('%d m lines' %(t // 1000000))
            #data_list.append(float(rating))
        if rating > 4.0:
            data_list.append(1)
        elif rating < 2.0:
            data_list.append(-1)
            count +=1
        else: 
            continue    
        
        user_list.append(int(word[0]))
        item_list.append(int(word[1]))
        
    print(count)
    users = list(set(user_list))
    items = list(set(item_list))
    user2id = dict(zip(users, range(len(users))))
    item2id = dict(zip(items, range(len(items))))

    user_id_list = [user2id[u] for u in user_list]
    item_id_list = [item2id[i] for i in item_list]

    result = spp.csr_matrix((data_list, (user_id_list, item_id_list)))
    return result

def extract_users(num_users, sparse_matrix):
	item_count = sparse_matrix.getnnz(axis = 1)
	user_sort = sorted(enumerate(item_count), key = lambda x: x[1], reverse = True)[:num_users]
	user_indices = [x[0] for x in user_sort]

	return spp.vstack([sparse_matrix.getrow(i) for i in user_indices])

def extract_items(num_items, sparse_matrix):
	user_count = sparse_matrix.getnnz(axis = 0)
	item_sort = sorted(enumerate(user_count), key = lambda x: x[1], reverse = True)[:num_items]
	item_indics = [x[0] for x in item_sort]

	return spp.hstack([sparse_matrix.getcol(j) for j in item_indics])

def get_reduced_matrix(num_users, num_items, filename):
	data = load_sparse_matrix(filename)
	data1 = extract_users(3 * num_users, data)
	data2 = extract_items(num_items, data1)
	data3 = extract_users(num_users, data2)
	return data3.toarray()

In [14]:
train_edges_ = np.load("./collab_users_entry_train.npy")
train_edges_.shape

(1015236, 3)

In [15]:
def construct_adjacency_matrix_from_G(G, num_users,num_items):
    num_nodes = num_users 
    A = spp.lil_matrix((num_nodes, num_nodes))
    for i in range(len(G)):
        user, item, weight = G[i]
        if weight == 1:
            A[user, item] = 1
        else: A[user , item] = -1
    return A
# G = np.load("GrQc_ALLusers_entry.npy")
A = construct_adjacency_matrix_from_G(train_edges_,235868,235868)

In [17]:
A_ = A.tocsr()
A.shape

(235868, 235868)

In [18]:
from sklearn.preprocessing import normalize

from scipy.sparse.linalg import svds

def ExtractFeatures(num, d, m):
    A1 = m[:num, :]  # giữ nguyên m là sparse matrix (CSR format)
    u, s, vt = svds(A1, k=d-1)  # sparse SVD
    u = normalize(u, axis=1, norm='l2')
    U = np.concatenate((u, np.ones((num, 1))), axis=1) / np.sqrt(2)
    return U

U = ExtractFeatures(235868, d=10, m = A)
np.save("./collab_users_items_features.npy", U)
I = ExtractFeatures(235868, d=10, m = A.T)
np.save("./collab_items_users_features.npy", I)
